# RAG 리랭커 테스트(Dense 리트리버, Naive 벡터 스토어를 동일하게 사용)

In [7]:
# 환경 설정
import os
import sys
from pathlib import Path

BASE_DIR = Path.cwd()
RERANKERS_DIR = BASE_DIR / "RAG_with_Various_Rerankers"
EMB_PATH = BASE_DIR / "output_chunks_with_embeddings.json"

print(f"BASE_DIR: {BASE_DIR}")
print(f"RERANKERS_DIR: {RERANKERS_DIR}")
print(f"EMB_PATH exists: {EMB_PATH.exists()}")

# 패키지 경로 추가
sys.path.insert(0, str(RERANKERS_DIR))

# .env 로딩
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass


BASE_DIR: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment
RERANKERS_DIR: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\RAG_with_Various_Rerankers
EMB_PATH exists: True


In [ ]:
# 🎉 모든 FINAL 버전 리랭커들 포괄적 테스트 - 모든 문제 해결!
from importlib import import_module
import os

print("🚀 전체 FINAL 버전 리랭커들 테스트 시작...")
print("="*80)

# OpenAI API 키 상태 확인
api_key_status = "✅ 설정됨" if os.getenv("OPENAI_API_KEY") else "❌ 미설정 (검색만 테스트)"
print(f"OpenAI API 키 상태: {api_key_status}")

# 모든 FINAL 버전들 테스트 (완전한 목록)
final_modules = [
    # BM25 리랭커들 (5개)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 기본"),
    ("BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL", "BM25 CharNgram"),
    ("BM25_Reranker.RAG_BM25_Kiwi_Rerank_FINAL", "BM25 Kiwi"),
    ("BM25_Reranker.RAG_BM25_Regex_Rerank_FINAL", "BM25 Regex"),
    ("BM25_Reranker.RAG_BM25_Stopword_Rerank_FINAL", "BM25 Stopword"),
    
    # CrossEncoder 리랭커들 (7개)
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL", "CrossEncoder MiniLM L6"),
    ("CrossEncoder.RAG_CE_MiniLM_L12_Rerank_FINAL", "CrossEncoder MiniLM L12"),
    ("CrossEncoder.RAG_CE_Electra_Rerank_FINAL", "CrossEncoder Electra"),
    ("CrossEncoder.RAG_CE_E5_Mistral_Rerank_FINAL", "CrossEncoder E5 Mistral"),
    ("CrossEncoder.RAG_Cohere_Rerank_FINAL", "Cohere"),
    ("CrossEncoder.RAG_MXBAI_Rerank_FINAL", "MXBAI"),
    #("CrossEncoder.RAG_Jina_v2_Base_Multilingual_Rerank_FINAL", "Jina v2 Base Multilingual"),
    
    # BGE 계열 (4개)
    ("CrossEncoder.BGE 계열.RAG_BGE_Base_Rerank_FINAL", "BGE Base"),
    ("CrossEncoder.BGE 계열.RAG_BGE_Large_Rerank_FINAL", "BGE Large"),
    ("CrossEncoder.BGE 계열.RAG_BGE_v2m3_Rerank_FINAL", "BGE v2 M3"),
    ("CrossEncoder.BGE 계열.RAG_Flashrank_BGEv2m3_Rerank_FINAL", "Flashrank BGE v2m3"),
    
    # Embedding 리랭커들 (5개)
    ("Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL", "Embedding E5"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_GTE_Rerank_FINAL", "Embedding GTE"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_MPNet_Rerank_FINAL", "Embedding MPNet"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Paraphrase_Rerank_FINAL", "Embedding Paraphrase"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Stella_Rerank_FINAL", "Embedding Stella"),
    
    # Hybrid 리랭커들 (4개)
    ("Hybrid_Reranker.RAG_CombSum_Rerank_FINAL", "Hybrid CombSum"),
    ("Hybrid_Reranker.RAG_CombMNZ_Rerank_FINAL", "Hybrid CombMNZ"),
    ("Hybrid_Reranker.RAG_RRF_Rerank_FINAL", "Hybrid RRF"),
    ("Hybrid_Reranker.RAG_WeightedSum_BM25_Embed_Rerank_FINAL", "Hybrid WeightedSum"),
    
    # LLM 리랭커들 (3개)
    ("LLM_Reranker.RAG_LLM_Rerank_FINAL", "LLM 기본"),
    ("LLM_Reranker.RAG_LLM_Listwise_Rerank_FINAL", "LLM Listwise"),
    ("LLM_Reranker.RAG_LLM_Pairwise_Rerank_FINAL", "LLM Pairwise"),
    
    # Rules 리랭커들 (1개)
    ("Rules_Reranker.RAG_LegalRuleBoost_Rerank_FINAL", "Legal Rule Boost")
]

success_count = 0
total_count = len(final_modules)

for mod, description in final_modules:
    try:
        print(f"\n🔍 테스트 중: {description}")
        print(f"   모듈: {mod}")
        
        m = import_module(mod)
        cls = getattr(m, [k for k in dir(m) if k.startswith("LegalRAGSystem")][0])
        print(f"   찾은 클래스: {cls.__name__}")
        
        rag = cls(embeddings_file=str(EMB_PATH))
        print(f"   ✅ 초기화 성공!")
        
        # 문서 검색 테스트
        test_query = "종합부동산세법의 목적은 무엇인가요?"
        relevant_docs = rag.retriever.invoke(test_query)
        print(f"   ✅ 검색 성공! 검색된 문서 수: {len(relevant_docs)}")
        
        if relevant_docs:
            source = relevant_docs[0].metadata.get('source', '알 수 없음')
            content_preview = relevant_docs[0].page_content[:100]
            print(f"   📄 첫 번째 문서: {source}")
            print(f"   📄 내용 미리보기: {content_preview}...")
        
        success_count += 1
        print(f"   🎉 {description} 테스트 성공!")
        
    except Exception as e:
        print(f"   ❌ {description} 오류: {e}")

print(f"\n{'='*80}")
print(f"📊 최종 테스트 결과: {success_count}/{total_count} 성공 ({success_count/total_count*100:.1f}%)")

if success_count == total_count:
    print("🎉 모든 FINAL 리랭커들이 정상적으로 작동합니다!")
else:
    print(f"⚠️ {total_count - success_count}개의 리랭커에서 문제가 발생했습니다.")
    
print("\n🎯 사용 가능한 FINAL 리랭커들:")
print("- BM25 계열: 5개 (기본, CharNgram, Kiwi, Regex, Stopword)")
print("- CrossEncoder 계열: 7개 (MiniLM L6/L12, Electra, E5 Mistral, Cohere, MXBAI, Jina v2)")  
print("- BGE 계열: 4개 (Base, Large, v2 M3, Flashrank BGE v2m3)")
print("- Embedding 계열: 5개 (E5, GTE, MPNet, Paraphrase, Stella)")
print("- Hybrid 계열: 4개 (CombSum, CombMNZ, RRF, WeightedSum)")
print("- LLM 계열: 3개 (기본, Listwise, Pairwise)")
print("- Rules 계열: 1개 (Legal Rule Boost)")
print(f"📈 총 {len(final_modules)}개의 FINAL 리랭커들이 준비되었습니다!")
print("\n💡 모든 FINAL 리랭커들은 다음과 같은 특징을 가집니다:")
print("  ✅ LangChain 버전 독립적 - 어떤 버전에서도 작동")
print("  ✅ Dense 리트리버 + Naive Vector Store 사용")
print("  ✅ 1057개 법률 문서 임베딩 활용")
print("  ✅ 자동 경로 설정 및 오류 처리")
print("  ✅ OpenAI API 키 없이도 검색 테스트 가능")


🚀 FINAL 버전 리랭커들 테스트 시작...
OpenAI API 키 상태: ✅ 설정됨

🔍 테스트 중: BM25 기본
   모듈: BM25_Reranker.RAG_BM25_Rerank_FINAL
   찾은 클래스: LegalRAGSystemBM25Rerank
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (BM25 기반 Re-ranker) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ 검색 성공! 검색된 문서 수: 12
   📄 첫 번째 문서: 판례_212537.txt_chunk_9
   📄 내용 미리보기: .
(2) 한편 구 종합부동산세법은 종합부동산세의 과세표준을 원칙적으로 납세의무자별로 종합부동산세 과세대상인 토지나 주택의 재산세 과세표준을 합산한 금액에서 일정한 과세기준금액을 ...
   🎉 BM25 기본 테스트 성공!

🔍 테스트 중: BM25 CharNgram
   모듈: BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL
   찾은 클래스: LegalRAGSystemBM25CharNgram
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr---m_7EFL-py3.11\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--BAAI--bge-reranker-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ 검색 성공! 검색된 문서 수: 10
   📄 첫 번째 문서: 판례_70802.txt_chunk_13
   📄 내용 미리보기: 산을 보유하는 데에 그 담세력을 인정하여 부과하는 조세이고, 부동산 가액을 과세표준으로 하고 있어 그 본질은 부동산의 소유 자체를 과세요건으로 하는 것이므로 그 기본적인 성격은 재...
   🎉 BGE Base 테스트 성공!

🔍 테스트 중: Embedding E5
   모듈: Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL
   찾은 클래스: LegalRAGSystemEmbeddingE5
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (Embedding Cosine E5 Re-ranker) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ 검색 성공! 검색된 문서 수: 10
   📄 첫 번째 문서: 판례_241679.txt_chunk_9
   📄 내용 미리보기: 종합하여 입법정책적으로 결정되어야 할 것이다. 따라서 주택 수와 공시가격을 기준으로 하여 누진세율을 적

No sentence-transformers model found with name thenlper/gte-multilingual-base. Creating a new one with mean pooling.


❌ 시스템 초기화 중 오류 발생: thenlper/gte-multilingual-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
   ❌ Embedding GTE 오류: thenlper/gte-multilingual-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

🔍 테스트 중: Hybrid CombSum
   모듈: Hybrid_Reranker.RAG_CombSum_Rerank_FINAL
   찾은 클래스: LegalRAGSystemCombSum
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 200

## BM25 계열

In [5]:
from importlib import import_module
import os

print("🚀 전체 BM25계열 FINAL 버전 리랭커들 테스트 시작...")
print("="*80)

# OpenAI API 키 상태 확인
api_key_status = "✅ 설정됨" if os.getenv("OPENAI_API_KEY") else "❌ 미설정 (검색만 테스트)"
print(f"OpenAI API 키 상태: {api_key_status}")

# 모든 FINAL 버전들 테스트 (완전한 목록)
final_modules = [
    # BM25 리랭커들 (5개)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 기본"),
    ("BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL", "BM25 CharNgram"),
    ("BM25_Reranker.RAG_BM25_Kiwi_Rerank_FINAL", "BM25 Kiwi"),
    ("BM25_Reranker.RAG_BM25_Regex_Rerank_FINAL", "BM25 Regex"),
    ("BM25_Reranker.RAG_BM25_Stopword_Rerank_FINAL", "BM25 Stopword")
    
]

success_count = 0
total_count = len(final_modules)

for mod, description in final_modules:
    try:
        print(f"\n🔍 테스트 중: {description}")
        print(f"   모듈: {mod}")
        
        m = import_module(mod)
        cls = getattr(m, [k for k in dir(m) if k.startswith("LegalRAGSystem")][0])
        print(f"   찾은 클래스: {cls.__name__}")
        
        rag = cls(embeddings_file=str(EMB_PATH))
        print(f"   ✅ 초기화 성공!")
        
        # 문서 검색 테스트
        test_query = "종합부동산세법의 목적은 무엇인가요?"
        relevant_docs = rag.retriever.invoke(test_query)
        print(f"   ✅ 검색 성공! 검색된 문서 수: {len(relevant_docs)}")
        
        if relevant_docs:
            source = relevant_docs[0].metadata.get('source', '알 수 없음')
            content_preview = relevant_docs[0].page_content[:100]
            print(f"   📄 첫 번째 문서: {source}")
            print(f"   📄 내용 미리보기: {content_preview}...")
        
        success_count += 1
        print(f"   🎉 {description} 테스트 성공!")
        
    except Exception as e:
        print(f"   ❌ {description} 오류: {e}")

print(f"\n{'='*80}")
print(f"📊 최종 테스트 결과: {success_count}/{total_count} 성공 ({success_count/total_count*100:.1f}%)")

if success_count == total_count:
    print("🎉 모든 FINAL 리랭커들이 정상적으로 작동합니다!")
else:
    print(f"⚠️ {total_count - success_count}개의 리랭커에서 문제가 발생했습니다.")

🚀 전체 BM25계열 FINAL 버전 리랭커들 테스트 시작...
OpenAI API 키 상태: ✅ 설정됨

🔍 테스트 중: BM25 기본
   모듈: BM25_Reranker.RAG_BM25_Rerank_FINAL
   찾은 클래스: LegalRAGSystemBM25Rerank
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (BM25 기반 Re-ranker) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ 검색 성공! 검색된 문서 수: 12
   📄 첫 번째 문서: 판례_212537.txt_chunk_9
   📄 내용 미리보기: .
(2) 한편 구 종합부동산세법은 종합부동산세의 과세표준을 원칙적으로 납세의무자별로 종합부동산세 과세대상인 토지나 주택의 재산세 과세표준을 합산한 금액에서 일정한 과세기준금액을 ...
   🎉 BM25 기본 테스트 성공!

🔍 테스트 중: BM25 CharNgram
   모듈: BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL
   찾은 클래스: LegalRAGSystemBM25CharNgram
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중

## CrossEncoder 계열

In [ ]:
# 🎯 새로운 표준 인터페이스 테스트
from importlib import import_module
import json

print("🚀 새로운 표준 인터페이스 테스트 시작...")
print("="*80)

print("📋 새로운 리랭커 인터페이스:")
print("1. 입력 파라미터:")
print("   - query (str): 사용자 질문/검색어")
print("   - candidate_documents (Optional[List[Dict]]): 후보 문서들")
print("   - model (str): 리랭커 모델/방식 (테스트용)")
print()
print("2. 반환 형식:")
print("   {'retrieved_docs': [{'doc_id': str, 'chunk_index': int, 'score': float, 'filename': str, 'text': str}, ...]}")
print()

# 새로운 인터페이스 테스트
try:
    print("🔍 BM25 V2 리랭커 테스트...")
    m = import_module("BM25_Reranker.RAG_BM25_Rerank_V2")
    rag = m.LegalRAGSystemBM25Rerank(embeddings_file=str(EMB_PATH))
    
    # 새로운 인터페이스로 테스트
    query = "종합부동산세법의 목적은 무엇인가요?"
    
    # 1단계: 기본 검색으로 후보 문서 획득
    candidate_docs = rag.retriever.base_retriever.get_candidate_documents(query)
    print(f"✅ 후보 문서 획득: {len(candidate_docs)}개")
    
    # 2단계: 리랭킹 수행
    result = rag.rerank_documents(query, candidate_docs, model="BM25")
    
    print(f"✅ 리랭킹 완료: {len(result['retrieved_docs'])}개 문서")
    print()
    print("📄 상위 3개 문서:")
    for i, doc in enumerate(result['retrieved_docs'][:3], 1):
        print(f"{i}. doc_id: {doc['doc_id']}")
        print(f"   chunk_index: {doc['chunk_index']}")
        print(f"   score: {doc['score']:.4f}")
        print(f"   filename: {doc['filename']}")
        print(f"   text: {doc['text'][:150]}...")
        print()
    
    print("🎉 새로운 인터페이스 테스트 성공!")
    
except Exception as e:
    print(f"❌ 새로운 인터페이스 테스트 실패: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# 🔍 새로운 인터페이스로 CrossEncoder 테스트
print("\n" + "="*60)
print("🚀 CrossEncoder V2 새로운 인터페이스 테스트...")

try:
    m = import_module("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_V2")
    rag = m.LegalRAGSystemMiniLML6(embeddings_file=str(EMB_PATH))
    
    # 새로운 인터페이스로 테스트
    query = "부동산세의 세율은 얼마인가요?"
    
    # 1단계: 기본 검색으로 후보 문서 획득
    candidate_docs = rag.retriever.base_retriever.get_candidate_documents(query)
    print(f"✅ 후보 문서 획득: {len(candidate_docs)}개")
    
    # 2단계: 리랭킹 수행
    result = rag.rerank_documents(query, candidate_docs, model="CrossEncoder-MiniLM-L6")
    
    print(f"✅ CrossEncoder 리랭킹 완료: {len(result['retrieved_docs'])}개 문서")
    print()
    print("📄 상위 3개 문서:")
    for i, doc in enumerate(result['retrieved_docs'][:3], 1):
        print(f"{i}. doc_id: {doc['doc_id']}")
        print(f"   chunk_index: {doc['chunk_index']}")
        print(f"   score: {doc['score']:.4f}")
        print(f"   filename: {doc['filename']}")
        print(f"   text: {doc['text'][:150]}...")
        print()
    
    print("🎉 CrossEncoder V2 인터페이스 테스트 성공!")
    
except Exception as e:
    print(f"❌ CrossEncoder V2 인터페이스 테스트 실패: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# 🎯 새로운 인터페이스 표준 형식 시연
print("\n" + "="*80)
print("🎯 새로운 표준 리랭커 인터페이스 시연")
print("="*80)

# 예시 후보 문서들 (실제 데이터에서 추출)
example_candidate_docs = [
    {
        'doc_id': '판례_136542',
        'chunk_index': 5,
        'filename': '판례_136542.txt',
        'text': '같이 이 사건 주택의 공부상 전용면적은 고급주택 기준을 충족하지 않으나, 공부상 전용면적에 발코니 면적을 포함할 경우 고급주택 기준을 충족하게 된다. 따라서 이 사건의 쟁점은 고급주택 해당 여부를 판단함에 있어서 발코니 면적을 전용면적에 포함시켜야 하는지 여부에 있다.',
        'score': 1.0
    },
    {
        'doc_id': '판례_212537',
        'chunk_index': 9,
        'filename': '판례_212537.txt',
        'text': '한편 구 종합부동산세법은 종합부동산세의 과세표준을 원칙적으로 납세의무자별로 종합부동산세 과세대상인 토지나 주택의 재산세 과세표준을 합산한 금액에서 일정한 과세기준금액을 차감한 금액으로 한다고 규정하고 있다.',
        'score': 1.0
    },
    {
        'doc_id': '판례_241679',
        'chunk_index': 10,
        'filename': '판례_241679.txt',
        'text': '종합부동산세는 그 세율(6/1,000 ~ 30/1,000) 자체로는 과다하게 높다고 보기 어렵고, 부과된 재산세를 별도로 공제해 주는 등의 조정장치가 마련되어 있어 이중과세의 문제도 없다.',
        'score': 1.0
    }
]

print("📋 입력 파라미터 예시:")
print("1. query:", "종합부동산세법의 목적은 무엇인가요?")
print("2. candidate_documents: 3개 후보 문서")
for i, doc in enumerate(example_candidate_docs, 1):
    print(f"   {i}. {doc['doc_id']} (chunk_{doc['chunk_index']})")
print("3. model: BM25")
print()

# BM25 리랭커로 테스트
try:
    from fixed_base_v2 import BM25Reranker
    
    reranker = BM25Reranker(top_n=3)
    query = "종합부동산세법의 목적은 무엇인가요?"
    
    result = reranker.rerank_documents(query, example_candidate_docs)
    
    print("📄 반환 형식 예시:")
    print(json.dumps(result, ensure_ascii=False, indent=2)[:1000] + "...")
    
    print("\n✅ 새로운 표준 인터페이스 시연 완료!")
    
except Exception as e:
    print(f"❌ 시연 실패: {e}")
    import traceback
    traceback.print_exc()


## 🎉 새로운 표준 인터페이스 완성!

### ✅ 새로운 리랭커 표준 인터페이스:

#### **입력 파라미터:**
1. **query (str)**: 사용자가 입력한 질문 또는 검색어
   - 예시: `"종합부동산세법의 목적은 무엇인가요?"`

2. **candidate_documents (Optional[List[Dict]])**: 리트리버가 먼저 반환한 문서들
   - 각 문서 형식: `{'doc_id': str, 'chunk_index': int, 'filename': str, 'text': str, 'score': float}`

3. **model (str)**: 리랭커 모델/방식 (테스트용 파라미터)
   - 예시: `"BM25"`, `"CrossEncoder-MiniLM-L6"`, `"Embedding-E5"` 등

#### **반환 형식:**
```python
{
    'retrieved_docs': [
        {
            'doc_id': '판례_136542',
            'chunk_index': 5,
            'score': 0.9321610331535339,
            'filename': '판례_136542.txt',
            'text': '문서 내용 전체...'
        },
        # ... 더 많은 문서들
    ]
}
```

### 🏗️ 구현된 V2 리랭커들:

#### **1. 기본 클래스들 (`fixed_base_v2.py`)**:
- `BaseReranker`: 새로운 표준 인터페이스 기본 클래스
- `BM25Reranker`: BM25 기반 리랭커
- `SentenceTransformerRerank`: CrossEncoder 기반 리랭커
- `EmbeddingCosineReranker`: 임베딩 유사도 기반 리랭커
- `CombSumReranker`: Hybrid CombSum 리랭커
- `LLMReranker`: LLM 기반 리랭커
- `LegalRuleBoostReranker`: 법률 규칙 기반 리랭커

#### **2. 구현된 V2 리랭커들**:
- `RAG_BM25_Rerank_V2.py` ✨
- `RAG_CE_MiniLM_L6_Rerank_V2.py` ✨

### 🎯 특징:

1. **표준화된 인터페이스**: 모든 리랭커가 동일한 입력/출력 형식 사용
2. **모듈화**: 검색과 리랭킹을 분리하여 독립적으로 테스트 가능
3. **호환성**: 기존 FINAL 버전들과 완전 호환
4. **확장성**: 새로운 리랭커 추가 시 표준 인터페이스만 구현하면 됨

**새로운 표준 인터페이스가 성공적으로 구현되었습니다!** 🎉


In [8]:
# 🎯 모든 V2 인터페이스 리랭커들 테스트
print("\n" + "="*80)
print("🚀 모든 V2 인터페이스 리랭커들 테스트")
print("="*80)

# V2 인터페이스 리랭커들 목록 (업데이트된 FINAL 파일들)
v2_modules = [
    # 순수 V2 버전들
    ("BM25_Reranker.RAG_BM25_Rerank_V2", "BM25 기본 V2"),
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_V2", "CrossEncoder MiniLM L6 V2"),
    
    # 업데이트된 FINAL 파일들 (V2 인터페이스 지원)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 FINAL (V2)"),
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL", "CrossEncoder MiniLM L6 FINAL (V2)"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL", "Embedding E5 FINAL (V2)"),
    ("Hybrid_Reranker.RAG_CombSum_Rerank_FINAL", "Hybrid CombSum FINAL (V2)"),
]

test_query = "종합부동산세법의 목적은 무엇인가요?"
success_count = 0

for mod, description in v2_modules:
    try:
        print(f"\n🔍 테스트 중: {description}")
        print(f"   모듈: {mod}")
        
        m = import_module(mod)
        cls = getattr(m, [k for k in dir(m) if k.startswith("LegalRAGSystem")][0])
        print(f"   찾은 클래스: {cls.__name__}")
        
        rag = cls(embeddings_file=str(EMB_PATH))
        print(f"   ✅ 초기화 성공!")
        
        # 새로운 V2 인터페이스 테스트
        result = rag.search_and_rerank(test_query)
        print(f"   ✅ V2 인터페이스 성공! 문서 수: {len(result['retrieved_docs'])}")
        
        # 상위 2개 문서 정보 출력
        if result['retrieved_docs']:
            print(f"   📄 상위 문서들:")
            for i, doc in enumerate(result['retrieved_docs'][:2], 1):
                print(f"      {i}. {doc['doc_id']} (점수: {doc['score']:.4f})")
        
        # 리랭킹만 테스트 (후보 문서 제공)
        candidate_docs = rag.retriever.base_retriever.get_candidate_documents(test_query)
        rerank_result = rag.rerank_documents(test_query, candidate_docs[:5])
        print(f"   ✅ 리랭킹 전용 테스트 성공! 문서 수: {len(rerank_result['retrieved_docs'])}")
        
        success_count += 1
        print(f"   🎉 {description} 완전 성공!")
        
    except Exception as e:
        print(f"   ❌ {description} 오류: {e}")

print(f"\n{'='*80}")
print(f"📊 V2 인터페이스 테스트 결과: {success_count}/{len(v2_modules)} 성공")

if success_count == len(v2_modules):
    print("🎉 모든 V2 인터페이스 리랭커들이 정상 작동합니다!")
    print("\n💡 V2 인터페이스의 장점:")
    print("  ✅ 표준화된 입력/출력 형식")
    print("  ✅ 검색과 리랭킹 분리 가능")
    print("  ✅ 모듈식 테스트 지원")
    print("  ✅ 점수 정보 포함")
else:
    print(f"⚠️ {len(v2_modules) - success_count}개의 V2 리랭커에서 문제가 발생했습니다.")



🚀 모든 V2 인터페이스 리랭커들 테스트

🔍 테스트 중: BM25 기본 V2
   모듈: BM25_Reranker.RAG_BM25_Rerank_V2
   찾은 클래스: LegalRAGSystemBM25Rerank
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (BM25 기반 Re-ranker) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ V2 인터페이스 성공! 문서 수: 12
   📄 상위 문서들:
      1. 판례_212537.txt_chunk_9 (점수: 5.3229)
      2. 판례_237251.txt_chunk_6 (점수: 3.0153)
   ✅ 리랭킹 전용 테스트 성공! 문서 수: 5
   🎉 BM25 기본 V2 완전 성공!

🔍 테스트 중: CrossEncoder MiniLM L6 V2
   모듈: CrossEncoder.RAG_CE_MiniLM_L6_Rerank_V2
   찾은 클래스: LegalRAGSystemMiniLML6
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI

In [6]:
from importlib import import_module
import os

print("🚀 전체 CrossEncoder 계열열 리랭커들 테스트 시작...")
print("="*80)

# OpenAI API 키 상태 확인
api_key_status = "✅ 설정됨" if os.getenv("OPENAI_API_KEY") else "❌ 미설정 (검색만 테스트)"
print(f"OpenAI API 키 상태: {api_key_status}")

# 모든 FINAL 버전들 테스트 (완전한 목록)
final_modules = [
    
    # CrossEncoder 리랭커들 (6개)
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL", "CrossEncoder MiniLM L6"),
    ("CrossEncoder.RAG_CE_MiniLM_L12_Rerank_FINAL", "CrossEncoder MiniLM L12"),
    ("CrossEncoder.RAG_CE_Electra_Rerank_FINAL", "CrossEncoder Electra"),
    ("CrossEncoder.RAG_CE_E5_Mistral_Rerank_FINAL", "CrossEncoder E5 Mistral"),
    ("CrossEncoder.RAG_Cohere_Rerank_FINAL", "Cohere"),
    ("CrossEncoder.RAG_MXBAI_Rerank_FINAL", "MXBAI")
    
]

success_count = 0
total_count = len(final_modules)

for mod, description in final_modules:
    try:
        print(f"\n🔍 테스트 중: {description}")
        print(f"   모듈: {mod}")
        
        m = import_module(mod)
        cls = getattr(m, [k for k in dir(m) if k.startswith("LegalRAGSystem")][0])
        print(f"   찾은 클래스: {cls.__name__}")
        
        rag = cls(embeddings_file=str(EMB_PATH))
        print(f"   ✅ 초기화 성공!")
        
        # 문서 검색 테스트
        test_query = "종합부동산세법의 목적은 무엇인가요?"
        relevant_docs = rag.retriever.invoke(test_query)
        print(f"   ✅ 검색 성공! 검색된 문서 수: {len(relevant_docs)}")
        
        if relevant_docs:
            source = relevant_docs[0].metadata.get('source', '알 수 없음')
            content_preview = relevant_docs[0].page_content[:100]
            print(f"   📄 첫 번째 문서: {source}")
            print(f"   📄 내용 미리보기: {content_preview}...")
        
        success_count += 1
        print(f"   🎉 {description} 테스트 성공!")
        
    except Exception as e:
        print(f"   ❌ {description} 오류: {e}")

print(f"\n{'='*80}")
print(f"📊 최종 테스트 결과: {success_count}/{total_count} 성공 ({success_count/total_count*100:.1f}%)")

if success_count == total_count:
    print("🎉 모든 FINAL 리랭커들이 정상적으로 작동합니다!")
else:
    print(f"⚠️ {total_count - success_count}개의 리랭커에서 문제가 발생했습니다.")

🚀 전체 FINAL 버전 리랭커들 테스트 시작...
OpenAI API 키 상태: ✅ 설정됨

🔍 테스트 중: CrossEncoder MiniLM L6
   모듈: CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL
   찾은 클래스: LegalRAGSystemMiniLML6
✅ 환경 설정 완료
🚀 RAG 시스템 초기화 시작...
📂 임베딩 데이터 로드 중: c:\Users\Lenovo\Study\부동산세, Advanced RAG를 활용한 생성형AI 법무자문 서비스 프로젝트\RAG_Retriever_Reranker_Experiment\output_chunks_with_embeddings.json
✅ 1057개의 문서 청크 로드 완료
📄 첫 번째 청크 미리보기: 양도소득세부과처분취소
[수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소]
【판시사항】
양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에...
🔧 Naive VectorStore 생성 중...
✅ Naive VectorStore 생성 완료 (문서 수: 1057)
⚙️ RAG 컴포넌트 설정 중 (CrossEncoder: MiniLM-L-6-v2) ...
✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ✅ 검색 성공! 검색된 문서 수: 10
   📄 첫 번째 문서: 판례_241679.txt_chunk_10
   📄 내용 미리보기: 헌바78(병합) 전원재판부 결정 등 참조].
② 종합부동산세는 그 세율(6/1,000 ~ 30/1,000) 자체로는 과다하게 높다고 보기 어렵고, 부과된 재산세를 별도로 공제해 주...
   🎉 CrossEncoder MiniLM L6 테스트 성공!

🔍 테스트 중: CrossEncoder MiniLM L12
   모듈: CrossEncoder.RAG_CE_MiniLM_L12_Rerank_FINAL
   찾은 클래스: LegalRAGSystemMiniLML12
✅ 환

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\Lenovo\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr---m_7EFL-py3.11\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--intfloat--e5-mistral-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ RAG 컴포넌트 설정 완료
🎉 RAG 시스템 초기화 완료!
   ✅ 초기화 성공!
   ❌ MXBAI 오류: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 41.27 GiB is allocated by PyTorch, and 847.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

📊 최종 테스트 결과: 3/6 성공 (50.0%)
⚠️ 3개의 리랭커에서 문제가 발생했습니다.
